# Introduction 


## How to use this notebook
This example runs Sima in locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note bookshould be run in the following way:
1. [Installation](#installation) can be run to install all the relevant tools, like python modules and local worker executable
2. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
3. [Run analysis](#run) shall be run each time a new Sima analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.

### Postprocessing
It is now possible to read the SIN/FEM file using SifIO, please consult the [Python examples](https://test.pypi.org/project/dnv-sifio/) C# [documentation](https://sesam.dnv.com/dev/api/sifio/). This notebook provides a small example script for getting node coordinates and displacements for the 200 first nodes for loadcase 11, [postprocessing.py](Workspace/CommonFiles/postprocessing.py). The script will be run just after Wasim and Sestra for each load case, see the [Run Wasim and Sestra](#runwasim) section. The section [Post processing](#postprocessing) shows how the post processed result files may be read and visualized in this notebook. More Python examples will be provided later. Relevant additional documents are:
- The input interface file documentations can be found [here](https://sesam.dnv.com/download/windows64/sesam_input_interface_format.pdf)
- The results interface format [here](https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf)



# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Set up custom user parameters <a id='custom'></a>
You need to set oneWorkflowTMPFolder to a folder that works for you. A short folder path  is recommended due to possible issues with Sima and long file names.

In [ ]:
from pathlib import Path
import os

oneWorkflowTMPFolder = r'C:\oneworkflowTmp' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorkflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorkflowTMPFolder)
        print(oneWorkflowTMPFolder + " created!\n")
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")

# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath
root_folder = os.getcwd()
print(root_folder)
workspacePath = str(Path(root_folder, 'Workspace'))
workspaceId = "SE28"
loadcase_file = f"{workspacePath}\\test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloudRun = False
clean = True
notebook_root_folder = os.getcwd()

## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [ ]:
from dnv.oneworkflow.utils.workunit_extension import *
from dnv.oneworkflow.utils import *
import shutil
#workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder,
#                                      local_workflow_runtime_temp_folders_cleanup=False,platform=Platform.LINUX, max_cores=5,environment=Environment.Testing,pool_id="SesamWorkflow_Linux_Standard_A2_v2")

workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder,
                                      local_workflow_runtime_temp_folders_cleanup=False,platform=Platform.WINDOWS, max_cores=5,environment=Environment.Testing,pool_id="SesamWorkflow_Windows_Standard_A2_v2")


# Run Sima analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [ ]:
from dnv.sesam.commands import SimaCommand
import pandas as pd
from dnv.onecompute.flowmodel import SchedulingOptions, FailureStrategy
"""Tests SIMA and Python commands"""
commands_info = []
if clean:
    shutil.rmtree(os.path.join(workspacePath,"LoadCases"), ignore_errors=True)
df_cases = pd.read_excel(loadcase_file, index_col=0)

for loadcase_folder_name, case in df_cases.iterrows():
    sima_cmd = SimaCommand().create_sima_command(task_name="WorkflowTask",workflow_name="ExampleWorkflow",sima_case_data=case.to_dict(),stask_filename=stask_file,stask_foldername=workflow_client.common_directory)
    commands_info.append(CommandInfo(commands=[sima_cmd],load_case_foldername=loadcase_folder_name))
print("Running commands in parallel")
await run_managed_commands_parallelly_async(
            client=workflow_client,
            scheduling_options=SchedulingOptions(failure_strategy=FailureStrategy.ContinueOnError),
            commands_info=commands_info,
            files_to_download_from_blob_to_client=FileOptions(max_size="11124MB",patterns=["**/*.log","**/*.txt", "**/*.lis", "**/*.MLG", "**/*.MLG","**/*.CSV"]),
            enable_common_files_copy_to_load_cases=True,
)
import time
time.sleep(5)


# Run Wasim and Sestra <a id='runwasim'></a>

In [ ]:
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
import json
import os
load_cases = ["test001", "test002"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def create_wasim_and_sestra_tasks(load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    commands_info = []
    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        casedict = case.to_dict()
        tasks = WasimAndSestraTaskCreator(casedict, additionalTemplateParameters).CreateTasks()
        post_processing_command = PythonCommand(
            directory=workflow_client.common_directory,
            filename="postprocessing.py")
        tasks.append(post_processing_command)
        cmd_info = CommandInfo(commands=tasks,load_case_foldername=casename)
        commands_info.append(cmd_info)
       

    return commands_info
print(os.getcwd())
print(workflow_client.workspace_path)


await run_managed_commands_parallelly_async(
            client=workflow_client,
            scheduling_options=SchedulingOptions(failure_strategy=FailureStrategy.ContinueOnError),
            commands_info=create_wasim_and_sestra_tasks(load_cases=load_cases),
            files_to_download_from_blob_to_client=FileOptions(max_size="11124MB",patterns=["**/*.txt", "**/*.lis", "**/*.MLG", "**/*.MLG","**/*.CSV"]),
            files_to_upload_from_client_to_blob=FileOptions(max_size="11124MB",patterns=["nofiles"]),
            enable_common_files_copy_to_load_cases=True,
)



# Post processing  <a id='postprocessing'></a>

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
print(workspacePath)
for folder in glob.glob(f"{workspacePath}\\LoadCases\\*"):
    test_name = folder.split("\\")[-1]
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)

def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(multiplot, resultcase=dataFrames.keys())
interactive_plot
